In [ ]:
import matplotlib.pyplot as plt
import pywt
import torchaudio as th_audio
import numpy as np
from math import sqrt, log
import matplotlib.pyplot as plt
from time import time

In [ ]:
signal, sr = th_audio.load("/home/samuel/Musique/Childish Gambino - Donald Glover Presents... {ALBUM}/05 - Beautiful.mp3")
signal = signal.mean(dim=0).numpy()[:256]

In [ ]:
signal.shape, sr

In [ ]:
t1 = time()

wt = "morl"
max_j = 32
scales = np.array([2 ** j for j in range(max_j)])
my_coefs, my_freqs = pywt.cwt(
    data=signal, scales=scales, wavelet=wt, sampling_period=1. / sr, method="conv"
)

t2 = time()

In [ ]:
print((t2 - t1) * 120 / 60, "h")

In [ ]:
my_coefs.shape, my_freqs.shape

In [ ]:
plt.matshow(my_coefs[:, :256])

In [ ]:
# https://github.com/PyWavelets/pywt/issues/328

mwf = pywt.ContinuousWavelet(wt, my_coefs.dtype).wavefun()
y_0 = mwf[0][np.argmin(np.abs(mwf[1]))]

def w_delta(s, n, wf, freqs):
    psi_hat_star = np.conjugate(np.fft.fft(wf[0]))
    return np.sum([psi_hat_star[np.argmin(
        np.abs(wf[0] - s * pywt.frequency2scale(wt, freqs[k]))
        )] for k in range(n)]) / n

def dj(s_arr):
    return (log(s_arr[1]) - log(s_arr[0])) / log(2)

kwargs = dict(
    n=len(scales),
    wf=mwf,
    freqs=my_freqs
)

d_j = dj(scales)
C_d = d_j / y_0 * np.sum([np.real(w_delta(s, **kwargs))/sqrt(s) for s in scales])

r_sum = np.transpose(np.sum(np.transpose(my_coefs)/ scales ** 0.5, axis=-1))
reconstructed = r_sum * (d_j / (C_d * y_0))

# finally add the mean of the data back on ('reconstructed' is an anomaly time series)
reconstructed += signal.mean()